Memento - Visualization of Inhabitance 

In [32]:
import pandas as pd
import plotly.express as px

# ----------------------------------------------------------
# Load data 
# ----------------------------------------------------------

#filename = '../data/memento_world_snapshot.small.csv'
filename = '../data/memento_world_snapshot.large.csv'
df = pd.read_csv(filename)

# absolute chunk coordinates
df['x'] = df['regionx'] * 32 + df['chunkx']
df['z'] = df['regionz'] * 32 + df['chunkz']



In [33]:
# ----------------------------------------------------------
# Verify integrity
# ----------------------------------------------------------

# Check 1: chunks should be unique by world, x, and z position
duplicate_chunks = df[df.duplicated(subset=['world', 'x', 'z'], keep=False)]

print(f"Total rows: {len(df)}")
print(f"Duplicate chunks (world, x, z): {len(duplicate_chunks)}")

if len(duplicate_chunks) > 0:
    print("\nDuplicate entries found:")
    print(duplicate_chunks[['world', 'x', 'z', 'timeinhabited_ticks']].sort_values(['world', 'x', 'z']))
else:
    print("\n✓ All chunks are unique by world, x, and z position")

Total rows: 1681
Duplicate chunks (world, x, z): 0

✓ All chunks are unique by world, x, and z position


In [34]:
# ----------------------------------------------------------
# Compute column-derived data
# ----------------------------------------------------------

# normalize inhabitance
df['inhabitance'] = (
    df['timeinhabited_ticks'] - df['timeinhabited_ticks'].min()
) / (
    df['timeinhabited_ticks'].max() - df['timeinhabited_ticks'].min()
)

In [35]:
# Create a collection of pivoted grids, one for each world
grids = {}
for world in df['world'].unique():
    world_data = df[df['world'] == world]
    grid = world_data.pivot_table(index='z', columns='x', values='inhabitance', aggfunc='first')
    grids[world] = grid
    print(f"{world}: {grid.shape[0]} rows × {grid.shape[1]} columns")

print(f"\nTotal worlds: {len(grids)}")
print(f"Grid keys: {list(grids.keys())}")

minecraft:overworld: 41 rows × 41 columns

Total worlds: 1
Grid keys: ['minecraft:overworld']


In [36]:
# Create a figure for each world's grid
figures = {}
for world, grid in grids.items():
    fig = px.imshow(
        grid,
        origin='lower',
        color_continuous_scale='YlOrRd',
        title=f'Minecraft Chunk Inhabitance Map - {world}',
        labels=dict(color='Inhabitance (normalized)')
    )
    
    fig.update_layout(
        xaxis_title='Chunk X',
        yaxis_title='Chunk Z',
        yaxis_scaleanchor='x'  # critical: square chunks
    )
    
    figures[world] = fig
    fig.show()

print(f"\nTotal figures: {len(figures)}")


Total figures: 1


In [37]:
# --- Define function to add grid and region borders to a figure ---

REGION_SIZE = 32

def add_grid_to_figure(fig, grid):
    """Add region borders and origin axes to a figure"""
    
    # raw grid extents (chunk coords)
    x_min_raw, x_max_raw = grid.columns.min(), grid.columns.max()
    z_min_raw, z_max_raw = grid.index.min(), grid.index.max()
    
    # expand to full region-aligned bounds
    x_min = (x_min_raw // REGION_SIZE) * REGION_SIZE
    x_max = ((x_max_raw // REGION_SIZE) + 1) * REGION_SIZE
    
    z_min = (z_min_raw // REGION_SIZE) * REGION_SIZE
    z_max = ((z_max_raw // REGION_SIZE) + 1) * REGION_SIZE
    
    # helper to generate region boundaries
    def region_lines(min_val, max_val, size):
        return range(int(min_val), int(max_val) + 1, size)
    
    # vertical region borders (X)
    for x in region_lines(x_min, x_max, REGION_SIZE):
        fig.add_shape(
            type="line",
            x0=x, x1=x,
            y0=z_min, y1=z_max,
            line=dict(color="rgba(0,0,0,0.35)", width=1),
            layer="above"
        )
    
    # horizontal region borders (Z)
    for z in region_lines(z_min, z_max, REGION_SIZE):
        fig.add_shape(
            type="line",
            x0=x_min, x1=x_max,
            y0=z, y1=z,
            line=dict(color="rgba(0,0,0,0.35)", width=1),
            layer="above"
        )
    
    # --- Emphasize origin axes (0,0) ---
    fig.add_shape(
        type="line",
        x0=0, x1=0,
        y0=z_min, y1=z_max,
        line=dict(color="black", width=2),
        layer="above"
    )
    
    fig.add_shape(
        type="line",
        x0=x_min, x1=x_max,
        y0=0, y1=0,
        line=dict(color="black", width=2),
        layer="above"
    )
    
    return fig

# Apply grid to each figure
for world, fig in figures.items():
    grid = grids[world]
    add_grid_to_figure(fig, grid)
    fig.show()